# StateLog × TradeLog 解析テンプレート
- `analysis/env_data` と `analysis/bt_results` に配置した CSV を読み込み、環境指標とBT結果を突き合わせるためのノートです。
- `merged_trades` DataFrame を作成しておけば、帯域別・時間別の集計を容易に反復できます。

In [132]:
import pandas as pd
from pathlib import Path
from functools import lru_cache


def find_workspace_root(marker='analysis'):
    current = Path.cwd().resolve()
    for candidate in [current, *current.parents]:
        if (candidate / marker).is_dir():
            return candidate
    raise FileNotFoundError("'{}' directory not found from {}".format(marker, current))


WORKSPACE_ROOT = find_workspace_root()
DATA_ROOT = WORKSPACE_ROOT / 'analysis'
ENV_DIR = DATA_ROOT / 'env_data'
BT_DIR = DATA_ROOT / 'bt_results'


def list_csv(directory, pattern='*.csv'):
    return sorted(directory.glob(pattern))


print('Workspace root:', WORKSPACE_ROOT)
print('Env dir:', ENV_DIR)
print('BT dir:', BT_DIR)
print('Detected env csv:', len(list_csv(ENV_DIR)))
print('Detected bt csv:', len(list_csv(BT_DIR, 'TradeLog_*.csv')))


def resolve_bt_path(bt_name=None, pattern='TradeLog_*.csv'):
    if bt_name:
        candidate = BT_DIR / bt_name
        if candidate.exists():
            return candidate
        raise FileNotFoundError(f"{candidate} が存在しません")
    files = list_csv(BT_DIR, pattern)
    if not files:
        raise FileNotFoundError(f"BT_DIR に {pattern} が見つかりません")
    return files[-1]


def load_bt_dataframe(bt_name=None):
    bt_path = resolve_bt_path(bt_name)
    df = pd.read_csv(bt_path)
    df['timestamp'] = pd.to_datetime(df['timestamp'].str.replace('.', '-', regex=False))
    return df, bt_path


@lru_cache(maxsize=1)
def build_env_index():
    env_map = {}
    for csv_path in list_csv(ENV_DIR, '*.csv'):
        digits = ''.join(filter(str.isdigit, csv_path.stem))
        if len(digits) < 8:
            continue
        key = digits[-8:]
        env_map.setdefault(key, []).append(csv_path)
    return {k: tuple(v) for k, v in env_map.items()}


def load_env_frames(date_keys, sep=';'):
    env_map = build_env_index()
    frames = []
    missing = []
    for key in date_keys:
        paths = env_map.get(key)
        if not paths:
            missing.append(key)
            continue
        day_frames = [pd.read_csv(path, sep=sep) for path in paths]
        env_part = pd.concat(day_frames, ignore_index=True)
        env_part['bar_time'] = pd.to_datetime(env_part['bar_time'].str.replace('.', '-', regex=False))
        frames.append(env_part)
    return frames, missing


def concat_env_frames(frames):
    if not frames:
        raise ValueError('frames is empty')
    return pd.concat(frames, ignore_index=True).sort_values('bar_time')


def merge_entries_with_env(bt_df, env_df, tolerance='5min'):
    entries = bt_df[bt_df['event'] == 'ENTRY'].copy().sort_values('timestamp')
    env_sorted = env_df.sort_values('bar_time')
    merged = pd.merge_asof(entries, env_sorted, left_on='timestamp', right_on='bar_time',
                           direction='nearest', tolerance=pd.Timedelta(tolerance))
    return merged


def build_entry_exit(bt_df, merged):
    exits = bt_df[bt_df['event'] == 'EXIT'][['ticket','net','pips','timestamp']].copy()
    exits = exits.rename(columns={'timestamp':'exit_timestamp','net':'exit_net','pips':'exit_pips'})
    entry_exit = merged.merge(exits, on='ticket', how='left')
    return entry_exit


def ensure_entry_exit():
    notebook_globals = globals()
    entry_exit = notebook_globals.get('entry_exit')
    if entry_exit is not None:
        return entry_exit
    bt_df = notebook_globals.get('bt_df')
    merged = notebook_globals.get('merged')
    if bt_df is None or merged is None:
        raise NameError('entry_exit を生成するには bt_df と merged が必要です。')
    entry_exit = build_entry_exit(bt_df, merged)
    notebook_globals['entry_exit'] = entry_exit
    return entry_exit

ATR_BUCKET_BINS = [0, 0.05, 0.08, 0.10, 0.12, 0.20, 1]
ATR_BUCKET_LABELS = ['0.00-0.05','0.05-0.08','0.08-0.10','0.10-0.12','0.12-0.20','0.20+']
ADX_BUCKET_BINS = [0, 15, 20, 25, 30, 100]
ADX_BUCKET_LABELS = ['<=15','15-20','20-25','25-30','30+']
DONCHIAN_BUCKET_BINS = [0, 0.15, 0.25, 0.4, 1]
DONCHIAN_BUCKET_LABELS = ['narrow','mid','wide','ultra']


Workspace root: /home/anyo_/workspace/YoYoEA_Multi_Entry
Env dir: /home/anyo_/workspace/YoYoEA_Multi_Entry/analysis/env_data
BT dir: /home/anyo_/workspace/YoYoEA_Multi_Entry/analysis/bt_results
Detected env csv: 229
Detected bt csv: 2


In [133]:
# ==== 自動で環境データを取得して突き合わせる ====
BT_FILE_NAME = None  # 解析対象のBTファイルを指定する場合はファイル名を入力

bt_df, bt_path = load_bt_dataframe(BT_FILE_NAME)
print(f'Using BT file: {bt_path.name} ({len(bt_df)} rows)')

date_keys = sorted(bt_df['timestamp'].dt.strftime('%Y%m%d').unique())
env_frames, missing_dates = load_env_frames(tuple(date_keys))
if missing_dates:
    print('[WARN] 環境ファイル未検出日:', ', '.join(missing_dates))

if not env_frames:
    raise FileNotFoundError('BT対象日に対応する環境ファイルが見つかりません')

env_df = concat_env_frames(env_frames)
print('Loaded env rows:', len(env_df), 'from', len(env_frames), '日分')

merged = merge_entries_with_env(bt_df, env_df)
print('merged rows', len(merged))
merged.head()


Using BT file: TradeLog_AtrBandConfig_YoYoEA_Multi_Entry_Test_AllOn_20250901_20251115.csv (7264 rows)
Loaded env rows: 15626 from 56 日分
merged rows 3632


,timestamp_x,run_id_x,event,symbol_x,profile_x,strategy,direction,ticket,volume,price,...,ma_long,ma_slope,bb_width,donchian_width,fibo_ratio,spread,session,weekday,volatility,notes
0,2025-09-01 01:05:00,20250901_010100,ENTRY,USDJPY,AtrBandConfig_YoYoEA_Multi_Entry_Test_AllOn_20...,MACD,SELL,1,0.1,146.946,...,147.031650,-0.102,0.0,0.199,0.497487,0.5,ASIA,1,LOW,NaN
1,2025-09-01 02:00:00,20250901_010100,ENTRY,USDJPY,AtrBandConfig_YoYoEA_Multi_Entry_Test_AllOn_20...,MACD,BUY,2,0.1,147.020,...,147.035225,0.187,0.0,0.177,0.661017,0.5,ASIA,1,LOW,NaN
2,2025-09-01 02:05:00,20250901_010100,ENTRY,USDJPY,AtrBandConfig_YoYoEA_Multi_Entry_Test_AllOn_20...,CCI,BUY,3,0.1,147.058,...,147.035770,0.252,0.0,0.178,0.764045,0.5,ASIA,1,LOW,NaN
3,2025-09-01 02:15:00,20250901_010100,ENTRY,USDJPY,AtrBandConfig_YoYoEA_Multi_Entry_Test_AllOn_20...,CCI,BUY,4,0.1,147.061,...,147.036615,0.295,0.0,0.178,0.752809,0.5,ASIA,1,LOW,NaN
4,2025-09-01 02:40:00,20250901_010100,ENTRY,USDJPY,AtrBandConfig_YoYoEA_Multi_Entry_Test_AllOn_20...,MA_CROSS,BUY,5,0.1,147.089,...,147.038675,0.365,0.0,0.178,0.837079,0.5,ASIA,1,LOW,NaN


In [134]:
# ==== ENTRY環境とEXIT結果を結合 ====
entry_exit = build_entry_exit(bt_df, merged)
print('combined rows', len(entry_exit))
entry_exit.head()


combined rows 3632


,timestamp_x,run_id_x,event,symbol_x,profile_x,strategy,direction,ticket,volume,price,...,donchian_width,fibo_ratio,spread,session,weekday,volatility,notes,exit_net,exit_pips,exit_timestamp
0,2025-09-01 01:05:00,20250901_010100,ENTRY,USDJPY,AtrBandConfig_YoYoEA_Multi_Entry_Test_AllOn_20...,MACD,SELL,1,0.1,146.946,...,0.199,0.497487,0.5,ASIA,1,LOW,NaN,-6.05,-8.9,2025-09-01 01:06:29
1,2025-09-01 02:00:00,20250901_010100,ENTRY,USDJPY,AtrBandConfig_YoYoEA_Multi_Entry_Test_AllOn_20...,MACD,BUY,2,0.1,147.020,...,0.177,0.661017,0.5,ASIA,1,LOW,NaN,2.04,3.0,2025-09-01 02:02:28
2,2025-09-01 02:05:00,20250901_010100,ENTRY,USDJPY,AtrBandConfig_YoYoEA_Multi_Entry_Test_AllOn_20...,CCI,BUY,3,0.1,147.058,...,0.178,0.764045,0.5,ASIA,1,LOW,NaN,2.99,4.4,2025-09-01 02:07:54
3,2025-09-01 02:15:00,20250901_010100,ENTRY,USDJPY,AtrBandConfig_YoYoEA_Multi_Entry_Test_AllOn_20...,CCI,BUY,4,0.1,147.061,...,0.178,0.752809,0.5,ASIA,1,LOW,NaN,2.04,3.0,2025-09-01 02:38:53
4,2025-09-01 02:40:00,20250901_010100,ENTRY,USDJPY,AtrBandConfig_YoYoEA_Multi_Entry_Test_AllOn_20...,MA_CROSS,BUY,5,0.1,147.089,...,0.178,0.837079,0.5,ASIA,1,LOW,NaN,4.07,6.0,2025-09-01 02:55:36


In [135]:
# ==== セッション×指標サマリ ====
entry_exit = ensure_entry_exit()
entry_exit['exit_net'] = entry_exit['exit_net'].astype(float)

if 'session' not in entry_exit.columns:
    entry_exit['session'] = 'UNKNOWN'
else:
    entry_exit['session'] = entry_exit['session'].fillna('UNKNOWN')

if 'strategy' not in entry_exit.columns:
    raise KeyError('strategy 列が存在しません')

summary_cols = ['session', 'strategy']


def summarize(group):
    total = len(group)
    wins = group['exit_net'] > 0
    losses = group['exit_net'] < 0
    gross_profit = group.loc[wins, 'exit_net'].sum()
    gross_loss = group.loc[losses, 'exit_net'].sum()
    profit_factor = gross_profit / abs(gross_loss) if gross_loss != 0 else float('inf')
    win_rate = wins.sum() / total if total else float('nan')
    avg_win = group.loc[wins, 'exit_net'].mean() if wins.any() else 0.0
    avg_loss = group.loc[losses, 'exit_net'].mean() if losses.any() else 0.0
    expectancy = group['exit_net'].mean()
    return pd.Series({
        'Trades': total,
        'Wins': int(wins.sum()),
        'Losses': int(losses.sum()),
        'Win Rate (%)': win_rate * 100,
        'PF': profit_factor,
        'Avg Win': avg_win,
        'Avg Loss': avg_loss,
        'Expectancy': expectancy,
    })

session_strategy = (
    entry_exit.groupby(summary_cols, group_keys=False, observed=False)
    .apply(summarize, include_groups=False)
    .reset_index()
    .sort_values(['session', 'strategy'])
)

format_dict = {
    'Win Rate (%)': '{:.2f}',
    'PF': '{:.2f}',
    'Avg Win': '{:.2f}',
    'Avg Loss': '{:.2f}',
    'Expectancy': '{:.2f}',
}

session_strategy.style.format(format_dict).set_table_attributes('style="table-layout: fixed; width: 100%;"')


,session,strategy,Trades,Wins,Losses,Win Rate (%),PF,Avg Win,Avg Loss,Expectancy
0,ASIA,CCI,503.000000,298.000000,205.000000,59.24,0.85,3.85,-6.60,-0.41
1,ASIA,MACD,252.000000,169.000000,83.000000,67.06,0.97,3.77,-7.90,-0.07
2,ASIA,MA_CROSS,49.000000,32.000000,17.000000,65.31,1.07,4.77,-8.37,0.21
3,ASIA,RSI,167.000000,109.000000,58.000000,65.27,1.00,4.41,-8.31,-0.01
4,ASIA,STOCH,113.000000,81.000000,32.000000,71.68,0.78,5.23,-16.87,-1.03
5,EUROPE,CCI,539.000000,353.000000,186.000000,65.49,0.98,4.50,-8.75,-0.07
6,EUROPE,MACD,258.000000,165.000000,93.000000,63.95,0.86,4.18,-8.60,-0.43
7,EUROPE,MA_CROSS,44.000000,31.000000,13.000000,70.45,1.47,5.73,-9.30,1.29
8,EUROPE,RSI,178.000000,104.000000,74.000000,58.43,0.90,5.34,-8.30,-0.33
9,EUROPE,STOCH,163.000000,113.000000,50.000000,69.33,1.14,6.32,-12.50,0.55


In [136]:
# ==== ATR×ADX帯域の損益集計 ====
entry_exit = ensure_entry_exit()
attr = entry_exit.dropna(subset=['atr_entry','adx14','strategy']).copy()
if attr.empty:
    raise ValueError('ATR/ADX/strategy が不足しています')
attr['atr_bucket'] = pd.cut(attr['atr_entry'].astype(float), bins=ATR_BUCKET_BINS, labels=ATR_BUCKET_LABELS, right=False)
attr['adx_bucket'] = pd.cut(attr['adx14'], bins=ADX_BUCKET_BINS, labels=ADX_BUCKET_LABELS, right=False)
metrics = attr.pivot_table(index=['atr_bucket','adx_bucket'], columns='strategy', values='exit_net', aggfunc=['count','sum','mean'], fill_value=0, observed=False)
idx = pd.IndexSlice
if 'count' in metrics.columns.get_level_values(0):
    metrics.loc[:, idx['count', :]] = metrics.loc[:, idx['count', :]].astype(int)
if 'sum' in metrics.columns.get_level_values(0):
    metrics.loc[:, idx['sum', :]] = metrics.loc[:, idx['sum', :]].round(2)
if 'mean' in metrics.columns.get_level_values(0):
    metrics.loc[:, idx['mean', :]] = metrics.loc[:, idx['mean', :]].round(2)
metrics.style     .format('{:.2f}', subset=idx[:, idx[['sum','mean'], :]])     .format('{:d}', subset=idx[:, idx[['count'], :]])     .set_table_attributes('style="table-layout: fixed; width: 100%;"')


In [137]:
# ==== ATR×ADX×ストラテジーの損益 ====
entry_exit = ensure_entry_exit()
attr = entry_exit.dropna(subset=['atr_entry','adx14','strategy']).copy()
attr['atr_bucket'] = pd.cut(attr['atr_entry'].astype(float), bins=ATR_BUCKET_BINS, labels=ATR_BUCKET_LABELS, right=False)
attr['adx_bucket'] = pd.cut(attr['adx14'], bins=ADX_BUCKET_BINS, labels=ADX_BUCKET_LABELS, right=False)
table = (
    attr.groupby(['atr_bucket','adx_bucket','strategy'], observed=False)
    .agg(count=('exit_net','size'), sum=('exit_net','sum'), mean=('exit_net','mean'))
    .reset_index()
)
table[['sum','mean']] = table[['sum','mean']].round(2)
table['count'] = table['count'].astype(int)
table.style.format({'count':'{:d}','sum':'{:.2f}','mean':'{:.2f}'})     .set_table_attributes('style="table-layout: fixed; width: 100%;"')


,atr_bucket,adx_bucket,strategy,count,sum,mean
0,0.00-0.05,<=15,CCI,9,-10.23,-1.14
1,0.00-0.05,<=15,MACD,9,3.88,0.43
2,0.00-0.05,<=15,MA_CROSS,1,-4.83,-4.83
3,0.00-0.05,<=15,RSI,0,0.00,nan
4,0.00-0.05,<=15,STOCH,0,0.00,nan
5,0.00-0.05,15-20,CCI,49,-14.37,-0.29
6,0.00-0.05,15-20,MACD,48,17.67,0.37
7,0.00-0.05,15-20,MA_CROSS,4,11.86,2.96
8,0.00-0.05,15-20,RSI,2,8.77,4.38
9,0.00-0.05,15-20,STOCH,14,-11.13,-0.80


In [138]:
# ==== Donchian×ATR×ストラテジーの損益 ====
entry_exit = ensure_entry_exit()
don = entry_exit.dropna(subset=['atr_entry','donchian_width','strategy']).copy()
don['atr_bucket'] = pd.cut(don['atr_entry'].astype(float), bins=ATR_BUCKET_BINS, labels=ATR_BUCKET_LABELS, right=False)
don['donchian_bucket'] = pd.cut(don['donchian_width'], bins=DONCHIAN_BUCKET_BINS, labels=DONCHIAN_BUCKET_LABELS, right=False)
table_d = don.pivot_table(index=['atr_bucket','donchian_bucket'], columns='strategy', values='exit_net', aggfunc=['count','sum','mean'], fill_value=0, observed=False)
idx = pd.IndexSlice
if 'count' in table_d.columns.get_level_values(0):
    table_d.loc[:, idx['count', :]] = table_d.loc[:, idx['count', :]].astype(int)
if 'sum' in table_d.columns.get_level_values(0):
    table_d.loc[:, idx['sum', :]] = table_d.loc[:, idx['sum', :]].round(2)
if 'mean' in table_d.columns.get_level_values(0):
    table_d.loc[:, idx['mean', :]] = table_d.loc[:, idx['mean', :]].round(2)
table_d.style     .format('{:.2f}', subset=idx[:, idx[['sum','mean'], :]])     .format('{:d}', subset=idx[:, idx[['count'], :]])     .set_table_attributes('style="table-layout: fixed; width: 100%;"')


In [139]:
# ==== 例: ATR帯域×戦略の損益集計 ====
exits = bt_df[bt_df['event'] == 'EXIT'].copy()
exits['atr_entry'] = exits['atr_entry'].astype(float)
bins = ATR_BUCKET_BINS
labels = ATR_BUCKET_LABELS
exits['atr_band'] = pd.cut(exits['atr_entry'], bins=bins, labels=labels, right=False)
pivot = exits.pivot_table(index='atr_band', columns='strategy', values='net', aggfunc='sum', fill_value=0, observed=False)
pivot = pivot.round(2)
pivot.style.format('{:.2f}').set_table_attributes('style="table-layout: fixed; width: 100%;"')


strategy,CCI,MACD,MA_CROSS,RSI,STOCH
atr_band,,,,,
0.00-0.05,-169.96,61.61,25.44,-43.01,-57.37
0.05-0.08,-189.52,-47.79,-48.23,-156.94,111.05
0.08-0.10,49.66,3.63,-13.06,-104.85,-212.31
0.10-0.12,-24.24,37.27,26.47,9.97,-68.90
0.12-0.20,-94.07,-22.99,-3.42,124.50,134.63
0.20+,40.77,7.40,41.18,0.00,-69.73


In [140]:
# ==== BT全体指標のサマリ ====
bt_exits = bt_df[bt_df['event'] == 'EXIT'].copy()
if bt_exits.empty:
    raise ValueError('EXITデータが存在しません')

bt_exits['net'] = bt_exits['net'].astype(float)
bt_exits['pips'] = bt_exits['pips'].astype(float)

total_trades = len(bt_exits)
wins = bt_exits['net'] > 0
losses = bt_exits['net'] < 0

gross_profit = bt_exits.loc[wins, 'net'].sum()
gross_loss = bt_exits.loc[losses, 'net'].sum()
profit_factor = gross_profit / abs(gross_loss) if gross_loss != 0 else float('inf')
win_rate = wins.sum() / total_trades if total_trades else float('nan')

avg_win = bt_exits.loc[wins, 'net'].mean() if wins.any() else 0.0
avg_loss = bt_exits.loc[losses, 'net'].mean() if losses.any() else 0.0
expectancy = bt_exits['net'].mean()

# 最大ドローダウンの算出
equity_curve = bt_exits['net'].cumsum()
equity_curve = pd.concat([pd.Series([0.0]), equity_curve], ignore_index=True)
rolling_peak = equity_curve.cummax()
drawdown = equity_curve - rolling_peak
max_drawdown = drawdown.min()
drawdown_pct = drawdown / rolling_peak.replace(0, pd.NA)
max_drawdown_pct = drawdown_pct.min()

summary = {
    'trades': total_trades,
    'wins': int(wins.sum()),
    'losses': int(losses.sum()),
    'win_rate_pct': win_rate * 100,
    'gross_profit': gross_profit,
    'gross_loss': gross_loss,
    'profit_factor': profit_factor,
    'avg_win': avg_win,
    'avg_loss': avg_loss,
    'expectancy': expectancy,
    'max_drawdown': max_drawdown,
    'max_drawdown_pct': max_drawdown_pct * 100,
}

label_map = {
    'trades': '総トレード数',
    'wins': '勝ち回数',
    'losses': '負け回数',
    'win_rate_pct': '勝率',
    'gross_profit': '総利益',
    'gross_loss': '総損失',
    'profit_factor': 'PF',
    'avg_win': '平均利益',
    'avg_loss': '平均損失',
    'expectancy': '期待値',
    'max_drawdown': '最大DD',
    'max_drawdown_pct': '最大DD(%)',
}
percent_keys = {'win_rate_pct', 'max_drawdown_pct'}
value_keys = {'profit_factor', 'avg_win', 'avg_loss', 'expectancy', 'gross_profit', 'gross_loss', 'max_drawdown'}

rows = []
for key in label_map:
    value = summary.get(key)
    if pd.isna(value):
        display_value = 'NaN'
    elif key in percent_keys:
        display_value = f"{value:.2f}%"
    elif key in value_keys:
        display_value = f"{value:.2f}"
    else:
        display_value = f"{int(value)}"
    rows.append({'指標': label_map[key], '値': display_value})

summary_df = pd.DataFrame(rows)
summary_df


,指標,値
0,総トレード数,3632
1,勝ち回数,2337
2,負け回数,1295
3,勝率,64.34%
4,総利益,11258.30
5,総損失,-11911.11
6,PF,0.95
7,平均利益,4.82
8,平均損失,-9.20
9,期待値,-0.18


In [141]:
# ==== 戦略別BT指標サマリ ====
bt_exits = bt_df[bt_df['event'] == 'EXIT'].copy()
if bt_exits.empty:
    raise ValueError('EXITデータが存在しません')
if 'strategy' not in bt_exits.columns:
    raise KeyError('strategy 列が存在しません')

bt_exits['net'] = bt_exits['net'].astype(float)

def summarize_group(group):
    total = len(group)
    wins = group['net'] > 0
    losses = group['net'] < 0
    gross_profit = group.loc[wins, 'net'].sum()
    gross_loss = group.loc[losses, 'net'].sum()
    profit_factor = gross_profit / abs(gross_loss) if gross_loss != 0 else float('inf')
    win_rate = wins.sum() / total if total else float('nan')
    avg_win = group.loc[wins, 'net'].mean() if wins.any() else 0.0
    avg_loss = group.loc[losses, 'net'].mean() if losses.any() else 0.0
    expectancy = group['net'].mean()
    return pd.Series({
        'trades': total,
        'wins': int(wins.sum()),
        'losses': int(losses.sum()),
        'win_rate_pct': win_rate * 100,
        'profit_factor': profit_factor,
        'avg_win': avg_win,
        'avg_loss': avg_loss,
        'expectancy': expectancy,
    })

strategy_summary = (
    bt_exits.groupby('strategy', group_keys=False)
    .apply(summarize_group)
    .reset_index()
)

rename_map = {
    'strategy': 'Strategy',
    'trades': 'Trades',
    'wins': 'Wins',
    'losses': 'Losses',
    'win_rate_pct': 'Win Rate (%)',
    'profit_factor': 'PF',
    'avg_win': 'Avg Win',
    'avg_loss': 'Avg Loss',
    'expectancy': 'Expectancy',
}
strategy_summary = strategy_summary.rename(columns=rename_map)

format_dict = {
    'Win Rate (%)': '{:.2f}',
    'PF': '{:.2f}',
    'Avg Win': '{:.2f}',
    'Avg Loss': '{:.2f}',
    'Expectancy': '{:.2f}',
}
strategy_summary.style.format(format_dict).hide(axis="index")


/tmp/ipykernel_171022/1671446051.py:34: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(summarize_group)


Strategy,Trades,Wins,Losses,Win Rate (%),PF,Avg Win,Avg Loss,Expectancy
CCI,1670.000000,1047.000000,623.000000,62.69,0.92,4.44,-8.08,-0.23
MACD,845.000000,558.000000,287.000000,66.04,1.02,4.36,-8.34,0.05
MA_CROSS,145.000000,93.000000,52.000000,64.14,1.05,6.14,-10.43,0.20
RSI,510.000000,316.000000,194.000000,61.96,0.90,4.99,-9.00,-0.33
STOCH,462.000000,323.000000,139.000000,69.91,0.93,6.29,-15.79,-0.35


## TODO
- 週次/日次での State×BT 結合関数を関数化する
- 追加の特徴量（ATR増減、セッションフラグなど）を派生列として定義する
- ML モデル用に `merged` から特徴量テーブルを作成する